In [13]:
# Created on Jan 29, 2020
# Autor: Alex Mottus
import pandas as pd # library to process data as dataframes

df_boroughes=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# Read in the first dataframe - read_html creates multiple dataframs
df_borough=df_boroughes[0]
# Remove records which have a 'Borough' value of 'Not Assigned'
df_borough.drop(df_borough[df_borough['Borough']=='Not assigned'].index, inplace=True)
# Loop through each row in the dataframe checking for Neighbourhoods with the value 'Not assigned'. \
# If one is found move over the value in the Borough cell
for index, row in df_borough.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']= row['Borough']
# Identify all records with the same Postal Code and Borough: merger the Neighbourhood values in a comma delimited format
df_b=df_borough.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
# Rename the postal code column
df_b.rename(columns={'Postcode':'Postalcode'}, inplace=True)

print(df_b.shape)

# Read in Geospactial data by Postal Code
df_data = pd.read_csv('http://cocl.us/Geospatial_data')
#print(df_data.head())
# My Code resumes here: Merge Dataframes based on the matching postal code fields
df_final=df_b.merge(df_data,how='inner', left_on='Postalcode', right_on='Postal Code')
# Remove extra postal code column
del df_final['Postal Code']
# Display top 10 Records
df_final.head(10)


(103, 3)


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
